### Simulate data
By simluating data, we can test the performance of our serving model and monitor the results via MLRun UI and the grafana dashboards. 

In [1]:
import mlrun
project_name = "ny-taxi-new-demo"
# get/create a project and register the data prep and trainer function in it
project = mlrun.get_or_create_project(
    name=project_name, context="./"
)

> 2023-02-16 15:41:56,846 [info] loaded project ny-taxi-new-demo from MLRun DB


In [2]:
serving_function = project.get_function("serving")

In [3]:
# testing the model with a single input
body = {'pickup_longitude':-73.844311, 'pickup_latitude':40.721319,
        'dropoff_longitude':-73.84161, 'dropoff_latitude': 40.712278,
        'passenger_count':1,
        'pickup_datetime': '2013-01-01T12', 'key': '2013-01-01T12'}
response = serving_function.invoke(path='/predict', body=body)
print(response['result_str'])

> 2023-02-16 15:42:02,433 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-ny-taxi-new-demo-serving.default-tenant.svc.cluster.local:8080/predict'}
predicted fare amount is 12.772906706331094


### Simulate incoming data

In [4]:

import numpy as np
# Load the dataset
data = mlrun.get_dataitem("https://s3.us-east-1.wasabisys.com/iguazio/data/nyc-taxi/test.csv").as_df()


In [5]:
from random import choice, uniform
from time import sleep
from tqdm.notebook import trange, tqdm

# Sending random requests
for i in trange(150):
    data_point = data.iloc[i].to_dict()
    try:
        resp = serving_function.invoke(path='/predict', body=data_point)
        print(resp)
        sleep(uniform(0.2, 0.03))
    except OSError:
        pass

  0%|          | 0/150 [00:00<?, ?it/s]

> 2023-02-16 15:42:08,310 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-ny-taxi-new-demo-serving.default-tenant.svc.cluster.local:8080/predict'}
{'result': 10.276585372412232, 'result_str': 'predicted fare amount is 10.276585372412232'}
> 2023-02-16 15:42:08,474 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-ny-taxi-new-demo-serving.default-tenant.svc.cluster.local:8080/predict'}
{'result': 11.458270912863185, 'result_str': 'predicted fare amount is 11.458270912863185'}
> 2023-02-16 15:42:08,668 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-ny-taxi-new-demo-serving.default-tenant.svc.cluster.local:8080/predict'}
{'result': 5.443648321045022, 'result_str': 'predicted fare amount is 5.443648321045022'}
> 2023-02-16 15:42:08,873 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-ny-taxi-new-demo-serving.default-tenant.svc.cluster.local:8080/predict'}
{'result': 7.310858273703016, 'result_str': 'predicted far